One Acre Fund: Loan Performance Scorecard Analysis

This notebook performs analysis on loan performance data with a focus on:
1. Loading and preprocessing the data
2. Taking a random sample of 5,000 loans to make analysis more manageable
3. Calculating key performance indicators
4. Visualizing repayment patterns and distributions
5. Identifying factors associated with loan repayment succes

In [1]:
import sys
sys.path.append('/Users/njui/Documents/Cline/MCP/jupyter-notebook-mcp/src')

from jupyter_ws_server import setup_jupyter_mcp_integration

# Start the WebSocket server
server, port = setup_jupyter_mcp_integration()

Loaded client.js from /Users/njui/Documents/Cline/MCP/jupyter-notebook-mcp/src/client.js
WebSocket server started on ws://localhost:8765


Jupyter client connected
External client connected (likely MCP server)


In [1]:
!which python

/Users/njui/Documents/dev_work/oaf/ds_assignmnt/.venv/bin/python


In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os
from datetime import datetime

# Set up plot styles
plt.style.use('seaborn-v0_8')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Add parent directory to Python path for imports
current_dir = Path(os.getcwd())
print(f"Current working directory: {current_dir}")

if current_dir.name == 'notebooks':
    project_dir = current_dir.parent.parent
else:
    project_dir = current_dir.parent

sys.path.append(str(project_dir))
print(f"Added {project_dir} to Python path")

Current working directory: /Users/njui/Documents/dev_work/oaf/ds_assignmnt/src/notebooks
Added /Users/njui/Documents/dev_work/oaf/ds_assignmnt to Python path


In [3]:
%load_ext jupyter_ai_magics


## Load and Preprocess Data

We'll load the training loan data, perform basic preprocessing, and take a random sample of 5,000 loans.

In [9]:
# Define helper functions
def load_data(file_path):
    """Load the loan data from CSV"""
    try:
        df = pd.read_csv(file_path, parse_dates=['contract_start_date'])
        print(f"Successfully loaded {len(df):,} rows from {file_path}")
        return df
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        print("Please ensure the file path is correct relative to your current directory")
        return None

def preprocess_data(df):
    """Perform preprocessing on loan data"""
    if df is None:
        return None
    
    # Create a copy to avoid modifying the original
    processed_df = df.copy()
    
    # Calculate deposit ratio
    processed_df['deposit_ratio'] = processed_df['deposit_amount'] / processed_df['nominal_contract_value']
    
    # Extract month and day information
    processed_df['month'] = processed_df['contract_start_date'].dt.month
    processed_df['contract_start_day'] = processed_df['contract_start_date'].dt.day
    processed_df['contract_day_name'] = processed_df['contract_start_date'].dt.day_name()
    processed_df['is_weekend'] = processed_df['contract_start_date'].dt.dayofweek >= 5
    
    # Calculate days since contract start
    reference_date = pd.Timestamp('2023-09-01')
    processed_df['days_since_start'] = (reference_date - processed_df['contract_start_date']).dt.days
    processed_df['months_since_start'] = (reference_date.year - processed_df['contract_start_date'].dt.year) * 12 + \
                                         (reference_date.month - processed_df['contract_start_date'].dt.month)
    
    # Calculate days to specific reference dates
    processed_df['days_diff_contract_start_to_sept_23'] = (pd.Timestamp('2023-09-01') - processed_df['contract_start_date']).dt.days
    processed_df['days_diff_contract_start_to_nov_23'] = (pd.Timestamp('2023-11-01') - processed_df['contract_start_date']).dt.days
    
    # Calculate month differences to specific reference dates
    processed_df['month_diff_contract_start_to_sept_23'] = ((pd.Timestamp('2023-09-15').year - processed_df['contract_start_date'].dt.year) * 12 + 
                                                           (pd.Timestamp('2023-09-15').month - processed_df['contract_start_date'].dt.month))
    processed_df['month_diff_contract_start_to_nov_23'] = ((pd.Timestamp('2023-11-15').year - processed_df['contract_start_date'].dt.year) * 12 + 
                                                          (pd.Timestamp('2023-11-15').month - processed_df['contract_start_date'].dt.month))
    
    # Calculate difference between November and September repayment rates
    processed_df['diff_nov_23_to_sept_23_repayment_rate'] = processed_df['nov_23_repayment_rate'] - processed_df['sept_23_repayment_rate']
    
    print(f"Preprocessing complete. Shape: {processed_df.shape}")
    return processed_df

In [11]:
# Attempt to load data with possible paths
possible_paths = [
    "../data/raw/training_loan_processed.csv",
    "../../data/raw/training_loan_processed.csv",
    "/Users/njui/Documents/dev_work/oaf/ds_assignmnt/data/raw/training_loan_processed.csv"
]

raw_data = None
for path in possible_paths:
    raw_data = load_data(path)
    if raw_data is not None:
        print(f"Successfully loaded data from {path}")
        break

if raw_data is None:
    print("\nCOULD NOT FIND DATA FILE. Please adjust the file path.")
    print("Current directory:", os.getcwd())
    print("Contents of current directory:", os.listdir())
else:
    # Print column names before preprocessing
    print("\nAvailable columns before preprocessing:")
    print(raw_data.columns.tolist())
    
    # Preprocess the data
    df = preprocess_data(raw_data)
    
    # Display basic information about the data
    print("\nDataset Info:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nSample data (first 5 rows):")
    display(df.head())

File not found: ../data/raw/training_loan_processed.csv
Please ensure the file path is correct relative to your current directory
Successfully loaded 36,215 rows from ../../data/raw/training_loan_processed.csv
Successfully loaded data from ../../data/raw/training_loan_processed.csv

Available columns before preprocessing:
['client_id', 'Loan_Type', 'region', 'duka_name', 'cumulative_amount_paid', 'nominal_contract_value', 'deposit_amount', 'contract_start_date', 'area', 'sales_territory', 'cumulative_amount_paid_start']


KeyError: 'nov_23_repayment_rate'

In [10]:
# Attempt to load data with possible paths
possible_paths = [
    "../data/raw/training_loan_processed.csv",
    "../../data/raw/training_loan_processed.csv",
    "/Users/njui/Documents/dev_work/oaf/ds_assignmnt/data/raw/training_loan_processed.csv"
]

raw_data = None
for path in possible_paths:
    raw_data = load_data(path)
    if raw_data is not None:
        print(f"Successfully loaded data from {path}")
        break

if raw_data is None:
    print("\nCOULD NOT FIND DATA FILE. Please adjust the file path.")
    print("Current directory:", os.getcwd())
    print("Contents of current directory:", os.listdir())
else:
    # Preprocess the data
    df = preprocess_data(raw_data)
    
    # Display basic information about the data
    print("\nDataset Info:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nSample data (first 5 rows):")
    display(df.head())

File not found: ../data/raw/training_loan_processed.csv
Please ensure the file path is correct relative to your current directory
Successfully loaded 36,215 rows from ../../data/raw/training_loan_processed.csv
Successfully loaded data from ../../data/raw/training_loan_processed.csv


KeyError: 'nov_23_repayment_rate'

In [6]:
# Load the training loan data file
data_path = "/Users/njui/Documents/dev_work/oaf/ds_assignmnt/data/raw/training_loan_processed.csv"

# Try to load the data
try:
    raw_data = pd.read_csv(data_path, parse_dates=['contract_start_date'])
    print(f"Successfully loaded {len(raw_data):,} rows from {data_path}")
    
    # Preprocess the data
    df = preprocess_data(raw_data)
    
    # Display basic information about the data
    print("\nDataset Info:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()[:10]} ... (showing first 10 columns)")
    print("\nSample data (first 3 rows):")
    display(df.head(3))
    
except FileNotFoundError:
    print(f"File not found: {data_path}")
    print("Please ensure the file path is correct relative to your current directory")
    print("Current directory:", os.getcwd())
    print("Contents of directory:", os.listdir())

Successfully loaded 36,215 rows from /Users/njui/Documents/dev_work/oaf/ds_assignmnt/data/raw/training_loan_processed.csv


KeyError: 'nov_23_repayment_rate'

## Take a Random Sample of 5,000 Loans

To make our analysis more manageable, we'll take a random sample of 5,000 loans while ensuring we maintain the distribution of key characteristics.

In [7]:
# Take a random sample of 5,000 loans
if 'df' in locals() and df is not None:
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Check original dataset size
    print(f"Original dataset size: {len(df):,} loans")
    
    # Sample size
    sample_size = min(5000, len(df))
    
    # Stratified sampling by loan type and region to maintain distribution
    sample_df = df.groupby(['Loan_Type', 'region']).apply(
        lambda x: x.sample(min(len(x), int(sample_size * len(x) / len(df))), random_state=42)
    ).reset_index(drop=True)
    
    # If we don't have enough samples, take a simple random sample
    if len(sample_df) < sample_size:
        additional_samples = df.drop(sample_df.index).sample(sample_size - len(sample_df), random_state=42)
        sample_df = pd.concat([sample_df, additional_samples])
    elif len(sample_df) > sample_size:
        sample_df = sample_df.sample(sample_size, random_state=42)
    
    print(f"Sample dataset size: {len(sample_df):,} loans")
    
    # Verify loan type distribution
    print("\nLoan Type Distribution - Original vs Sample:")
    orig_loan_dist = df['Loan_Type'].value_counts(normalize=True).sort_index()
    sample_loan_dist = sample_df['Loan_Type'].value_counts(normalize=True).sort_index()
    compare_df = pd.DataFrame({
        'Original': orig_loan_dist,
        'Sample': sample_loan_dist
    })
    display(compare_df)
    
    # Verify region distribution
    print("\nRegion Distribution - Original vs Sample:")
    orig_region_dist = df['region'].value_counts(normalize=True).sort_index()
    sample_region_dist = sample_df['region'].value_counts(normalize=True).sort_index()
    compare_region_df = pd.DataFrame({
        'Original': orig_region_dist,
        'Sample': sample_region_dist
    })
    display(compare_region_df)
    
    # Use the sample for further analysis
    df = sample_df
    print("\nUsing 5,000 sample dataset for further analysis")
else:
    print("Cannot create sample - dataset not loaded properly")

Cannot create sample - dataset not loaded properly


## Summary Statistics and Key Performance Indicators

Let's calculate and display key performance indicators for our loan portfolio.

In [8]:
def get_summary_statistics(df):
    """Calculate summary statistics for the loan portfolio"""
    if df is None:
        return None
    
    # Exchange rate KES to USD (approximate)
    kes_to_usd = 130
    
    stats = {
        # Portfolio size
        'loan_count': len(df),
        'total_portfolio_kes': df['nominal_contract_value'].sum(),
        'total_portfolio_usd': df['nominal_contract_value'].sum() / kes_to_usd,
        
        # Loan values
        'avg_loan_value': df['nominal_contract_value'].mean(),
        'avg_loan_value_usd': df['nominal_contract_value'].mean() / kes_to_usd,
        'median_loan_value': df['nominal_contract_value'].median(),
        'median_loan_value_usd': df['nominal_contract_value'].median() / kes_to_usd,
        
        # Repayment performance
        'avg_repayment_rate': df['nov_23_repayment_rate'].mean(),
        'target_achievement_rate': (df['nov_23_repayment_rate'] >= 0.98).mean(),
        'median_deposit_ratio': df['deposit_ratio'].median(),
        
        # Portfolio composition
        'loan_type_counts': df['Loan_Type'].value_counts().to_dict()
    }
    
    return stats

if 'df' in locals() and df is not None:
    # Calculate summary statistics
    stats = get_summary_statistics(df)
    
    print("\nPortfolio Overview:")
    print(f"Total Portfolio Value: KES {stats['total_portfolio_kes']:,.0f} (USD {stats['total_portfolio_usd']:,.0f})")
    print(f"Total Loans: {stats['loan_count']:,}")

    print("\nLoan Values:")
    print(f"Average Loan: KES {stats['avg_loan_value']:,.0f} (USD {stats['avg_loan_value_usd']:,.0f})")
    print(f"Median Loan: KES {stats['median_loan_value']:,.0f} (USD {stats['median_loan_value_usd']:,.0f})")

    print("\nRepayment Performance:")
    print(f"Average Rate: {stats['avg_repayment_rate']:.1%}")
    print(f"Target Achievement: {stats['target_achievement_rate']:.1%}")
    print(f"Median Deposit Ratio: {stats['median_deposit_ratio']:.1%}")

    print("\nPortfolio Composition:")
    for loan_type, count in stats['loan_type_counts'].items():
        print(f"- {loan_type}: {count:,} loans")
else:
    print("Cannot calculate statistics - dataset not loaded properly")

Cannot calculate statistics - dataset not loaded properly


## Repayment Distribution Analysis

Compare the distribution of repayment rates between September and November 2023.

In [9]:
if 'df' in locals() and df is not None:
    # Calculate repayment statistics for September
    sept_stats = {
        'mean': df['sept_23_repayment_rate'].mean(),
        'median': df['sept_23_repayment_rate'].median(),
        'target_achieved': (df['sept_23_repayment_rate'] >= 0.98).mean(),
        'count': len(df)
    }

    # Calculate repayment statistics for November
    nov_stats = {
        'mean': df['nov_23_repayment_rate'].mean(),
        'median': df['nov_23_repayment_rate'].median(),
        'target_achieved': (df['nov_23_repayment_rate'] >= 0.98).mean(),
        'count': len(df)
    }
    
    print("\nRepayment Statistics Comparison:")
    print(f"September - Mean: {sept_stats['mean']:.1%}, Median: {sept_stats['median']:.1%}, Target Achieved: {sept_stats['target_achieved']:.1%}")
    print(f"November - Mean: {nov_stats['mean']:.1%}, Median: {nov_stats['median']:.1%}, Target Achieved: {nov_stats['target_achieved']:.1%}")
    print(f"Improvement in Target Achievement: {nov_stats['target_achieved'] - sept_stats['target_achieved']:.1%}")
    
    # Plot repayment distributions
    plt.figure(figsize=(12, 6))
    
    # Create bins for histogram
    bins = np.linspace(0, 1, 21)  # 20 bins from 0 to 1
    
    # Plot September distribution
    plt.hist(df['sept_23_repayment_rate'], bins=bins, alpha=0.5, label='September 2023', color='blue')
    
    # Plot November distribution
    plt.hist(df['nov_23_repayment_rate'], bins=bins, alpha=0.5, label='November 2023', color='green')
    
    # Add reference line at 98% target
    plt.axvline(0.98, color='red', linestyle='--', label='98% Target')
    
    # Format plot
    plt.xlabel('Repayment Rate')
    plt.ylabel('Number of Loans')
    plt.title('Distribution of Repayment Rates: September vs November 2023')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
else:
    print("Cannot generate plots - dataset not loaded properly")

Cannot generate plots - dataset not loaded properly


## Repayment Curves Analysis

Analyze how repayment rates change over time from contract start date.

In [10]:
def plot_repayment_curve_with_cure_rates(df, overall_progress, day_cure_rates, stats):
    """Plot repayment curve with cure rates by contract start day"""
    if df is None or overall_progress is None or day_cure_rates is None or stats is None:
        return None
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Plot 1: Repayment progression over time
    ax1.plot(overall_progress['bin_mid'], overall_progress['nov_23_repayment_rate'], 
             marker='o', linestyle='-', linewidth=2, markersize=8, color='darkgreen',
             label='All Loans')
    
    # Add reference lines
    ax1.axhline(y=0.98, color='red', linestyle='--', label='98% Target')
    ax1.axvline(x=200, color='red', linestyle='--', label='September 2023')
    ax1.axvline(x=250, color='blue', linestyle='--', label='November 2023')
    
    # Add contract day lines (select a few for clarity)
    days_to_plot = [1, 10, 20, 30]
    colors = ['darkblue', 'darkgreen', 'darkorange', 'darkred']
    
    for day, color in zip(days_to_plot, colors):
        day_df = df[df['contract_start_day'] == day]
        if len(day_df) > 0:
            day_bins = np.arange(0, 301, 10)
            day_df['days_bin'] = pd.cut(day_df['days_diff_contract_start_to_nov_23'], bins=day_bins)
            day_progress = day_df.groupby('days_bin')['nov_23_repayment_rate'].mean().reset_index()
            day_progress['bin_mid'] = day_progress['days_bin'].apply(lambda x: x.mid)
            
            ax1.plot(day_progress['bin_mid'], day_progress['nov_23_repayment_rate'], 
                     linestyle='--', alpha=0.7, color=color, label=f'Day {day}')
    
    # Format plot 1
    ax1.set_xlabel('Days Since Contract Start')
    ax1.set_ylabel('Repayment Rate')
    ax1.set_title('Loan Repayment Progression Over Time')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper left')
    ax1.set_ylim(0, 1.05)
    ax1.set_xlim(0, 300)
    
    # Add annotations
    stats_text = f"""Contracts below 98% in September: {stats['pct_below_target_sept']:.1f}%
Contracts below 98% in November: {stats['pct_below_target_nov']:.1f}%
Contracts that cured (reached 98%): {stats['pct_cured']:.1f}%"""
    
    ax1.text(0.05, 0.05, stats_text, transform=ax1.transAxes, 
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
             verticalalignment='bottom')
    
    # Plot 2: Cure rates by contract start day
    bars = ax2.bar(day_cure_rates['contract_start_day'], day_cure_rates['cure_rate'], 
                  color='cornflowerblue', alpha=0.7)
    
    # Highlight overall cure rate
    ax2.axhline(y=stats['overall_cure_rate'], color='green', linestyle='-', 
               label=f"Overall: {stats['overall_cure_rate']:.1%}")
    
    # Highlight median cure rate
    ax2.axhline(y=stats['median_cure_rate'], color='black', linestyle='--', 
               label=f"Median: {stats['median_cure_rate']:.1%}")
    
    # Format plot 2
    ax2.set_xlabel('Contract Start Day')
    ax2.set_ylabel('Cure Rate (Nov - Sept)')
    ax2.set_title('Cure Rate by Contract Start Day')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.legend(loc='upper right')
    
    # Add stats text
    cure_stats = f"""Cure Rate Statistics:
Min: {stats['min_cure_rate']:.1%} (Day {stats['min_cure_day']})
Median: {stats['median_cure_rate']:.1%}
Max: {stats['max_cure_rate']:.1%} (Day {stats['max_cure_day']})
Overall: {stats['overall_cure_rate']:.1%}"""
    
    ax2.text(0.95, 0.05, cure_stats, transform=ax2.transAxes,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
             verticalalignment='bottom', horizontalalignment='right')
    
    # Adjust layout and return figure
    plt.tight_layout()
    return fig

def analyze_repayment_curves(df):
    """Analyze repayment patterns over time"""
    if df is None:
        return None, None, None

    # Create bins based on days since contract start
    bins = np.arange(0, 301, 10)  # From 0 to 300 days in steps of 10
    df['days_bin'] = pd.cut(df['days_diff_contract_start_to_nov_23'], bins=bins)
    
    # Calculate overall repayment progress by bin
    overall_progress = df.groupby('days_bin')['nov_23_repayment_rate'].mean().reset_index()
    overall_progress['bin_mid'] = overall_progress['days_bin'].apply(lambda x: x.mid)
    
    # Calculate cure rates by contract start day (which day of month the contract started)
    day_cure_rates = df.groupby('contract_start_day')['nov_23_repayment_rate'].mean().reset_index()
    day_cure_rates.columns = ['contract_start_day', 'cure_rate']
    
    # Calculate statistics
    stats = {
        'pct_below_target_sept': (df['sept_23_repayment_rate'] < 0.98).mean() * 100,
        'pct_below_target_nov': (df['nov_23_repayment_rate'] < 0.98).mean() * 100,
        'pct_cured': ((df['sept_23_repayment_rate'] < 0.98) & (df['nov_23_repayment_rate'] >= 0.98)).mean() * 100,
        'min_cure_rate': day_cure_rates['cure_rate'].min(),
        'min_cure_day': day_cure_rates.loc[day_cure_rates['cure_rate'].idxmin(), 'contract_start_day'],
        'max_cure_rate': day_cure_rates['cure_rate'].max(),
        'max_cure_day': day_cure_rates.loc[day_cure_rates['cure_rate'].idxmax(), 'contract_start_day'],
        'median_cure_rate': day_cure_rates['cure_rate'].median(),
        'overall_cure_rate': day_cure_rates['cure_rate'].mean()
    }
    
    return overall_progress, day_cure_rates, stats

# Analyze repayment curves if data is loaded
if 'df' in locals() and df is not None:
    # Analyze repayment curves by day
    overall_progress, day_cure_rates, stats = analyze_repayment_curves(df)
    
    # Plot the results
    fig = plot_repayment_curve_with_cure_rates(df, overall_progress, day_cure_rates, stats)
    plt.show()
else:
    print("Cannot analyze repayment curves - dataset not loaded properly")

Cannot analyze repayment curves - dataset not loaded properly


## Repayment by Day of Week

Analyze how repayment rates vary by the day of week the contract was started.